In [1]:
# importing libraries
from keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding

from transformers import BertTokenizer, TFBertForSequenceClassification, AdamWeightDecay
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential
import tensorflow as tf

# Load the TextPreprocessor class (assumed to be defined already)
from textpreprocessor import TextPreprocessor
import warnings
warnings.filterwarnings("ignore")

2024-09-22 22:06:18.734431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 22:06:18.742828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 22:06:18.745296: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 22:06:18.751802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-22 22:06:19.138798: W tensorflow/compiler/tf2

In [2]:
# Initialize the Text Preprocessor
processor = TextPreprocessor()

# Load data
df_train, df_test = processor.load_data(num_sample=10000, test_ratio=0.2)

# Preprocess data
df_train = processor.preprocess(df_train)
df_test = processor.preprocess(df_test)

# Split data
X_train, y_train = processor.split_data(df_train)
X_test, y_test = processor.split_data(df_test)

Sampled Data Shape:
(10000, 3)
Sampled Data Shape:
(2000, 3)


In [3]:
display(X_train.shape)
display(X_test.shape)

(9999,)

(2000,)

In [4]:
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train)

In [5]:
# # using tokenizer to transform text messages into training and testing set
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# X_test_seq = tokenizer.texts_to_sequences(X_test)

In [6]:
# X_train_seq_padded = pad_sequences(X_train_seq, maxlen=200)
# X_test_seq_padded = pad_sequences(X_test_seq, maxlen=200)

In [7]:
X_train_seq_padded, X_test_seq_padded = processor.tokenization_and_padding(X_train, X_test)

In [8]:
X_train_seq_padded[0]

array([   0,    8,   85, 4021,    7,    4,  391,    7, 1026, 4904,  564,
       3252,    2,  556, 4022,  784,  388,  172,  716,   22,  307,  602,
         18,  103,   46,  293,  123, 3252,   13,   27,   23,  304,    2,
         13, 3088,   73,    1,   49,    3,  248,    5,    1, 3252,  229,
        646,  242,  117,    2,    1, 2848,   33,    1, 4583,  170,  556,
       4022,  784,   13,  542,  382, 2959,    2,    6,  881,  117,   10,
        274,   73,  172,  769,  217,    1,  180,    9,    4,    3,   90,
       1870,   19,  203,   83,    9,  850,    5,   88, 1349,   46, 2143,
          2, 1562,  805,  722,    3,   90,  864,  182,   70, 2960,    8,
        742], dtype=int32)

In [9]:
tokenizer = processor.tokenizer

In [10]:
# construct model
BATCH_SIZE = 64

model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1,64))
model.add(Bidirectional(LSTM(100, dropout=0,recurrent_dropout=0)))
model.add(Dense(128, activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile("adam","binary_crossentropy",metrics=["accuracy"])
model.summary()

I0000 00:00:1727035589.591355   24725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727035589.592003   24725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727035589.592029   24725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727035589.594726   24725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727035589.594756   24725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# # Used for preventing ovefitting
# from keras.callbacks import EarlyStopping
# early_stop = EarlyStopping(monitor="val_loss",patience=5,verbose=True)

In [12]:
history = model.fit(X_train_seq_padded, y_train,batch_size=BATCH_SIZE,epochs=15,
                    validation_data=(X_test_seq_padded, y_test))

Epoch 1/15


2024-09-22 22:06:31.359904: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.6243 - loss: 0.6320 - val_accuracy: 0.7975 - val_loss: 0.4334
Epoch 2/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.8579 - loss: 0.3272 - val_accuracy: 0.8355 - val_loss: 0.3850
Epoch 3/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9157 - loss: 0.2212 - val_accuracy: 0.8235 - val_loss: 0.4404
Epoch 4/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9416 - loss: 0.1679 - val_accuracy: 0.8295 - val_loss: 0.5656
Epoch 5/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9514 - loss: 0.1251 - val_accuracy: 0.8215 - val_loss: 0.5829
Epoch 6/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9760 - loss: 0.0764 - val_accuracy: 0.7850 - val_loss: 0.7934
Epoch 7/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9773 - loss: 0.0691 - val_accuracy: 0.8035 - val_loss: 0.7178
Epoch 8/15
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9867 - loss: 0.0463 - val_accuracy: 0.805

In [13]:
from sklearn.metrics import roc_auc_score
pred_train = model.predict(X_train_seq_padded)
pred_test = model.predict(X_test_seq_padded)
print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(y_train, pred_train)))
print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(y_test, pred_test)))

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
LSTM Recurrent Neural Network baseline: 0.9999036927643138
LSTM Recurrent Neural Network: 0.8599638391713629


In [14]:
model.evaluate(X_test_seq_padded, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8019 - loss: 1.2576


[1.248485803604126, 0.7919999957084656]

In [15]:
acc = history.history["accuracy"]
loss = history.history["loss"]

val_acc = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

# plt.figure(figsize=(9,6))
# plt.plot(acc,label="Training Accuracy")
# plt.plot(val_acc,label="Validation Accuracy")
# plt.legend()
# plt.ylabel("Accuracy")
# plt.title("Training and Validation Accuracy")

In [16]:
# plt.figure(figsize=(9,6))
# plt.plot(loss,label="Training Loss")
# plt.plot(val_loss, label="Validation Loss")
# plt.legend()
# plt.ylabel("Loss")
# plt.title("Training and Validation Loss")

In [22]:
# Evaluate the model
y_pred_prob = model.predict(X_test_seq_padded)
y_pred = (y_pred_prob > 0.5).astype("int32")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [26]:
# Step 7: Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 79.20%
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.75      0.78       976
           1       0.78      0.83      0.80      1024

    accuracy                           0.79      2000
   macro avg       0.79      0.79      0.79      2000
weighted avg       0.79      0.79      0.79      2000



In [17]:
# model.save("BiLSTM1.h5")